In [1]:
CLIENT_ID = "0eZo-Z0nv8M0uw" 
CLIENT_SECRET = "jn0jrTlgJ-yhYh0yyBAcVFBzc9g"

In [2]:
USER_AGENT = "python:<test0.1> (by /u/abhijit)"

In [3]:
USERNAME = "abhijit-tambe" 
PASSWORD = " "

In [4]:
import requests
def login(username, password):
    if password is None:
        password = getpass.getpass("Enter reddit password for user {}: ".format(username))    
    headers = {"User-Agent": USER_AGENT}
    # Setup an auth object with our credentials
    client_auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
    # Make a post request to the access_token endpoint
    post_data = {"grant_type": "password", "username": username, "password": password}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth,     
                             data=post_data, headers=headers) 
    return response.json()

In [28]:
token = login(USERNAME, PASSWORD)


In [6]:
token

{'access_token': '179517556424-qtvHrtkLTlWE79ZArhHc-2y2YTs',
 'expires_in': 3600,
 'scope': '*',
 'token_type': 'bearer'}

In [7]:
subreddit = "worldnews"

In [8]:
url = "https://oauth.reddit.com/r/{}".format(subreddit)

In [9]:
headers = {"Authorization": "bearer {}".format(token['access_token']), 
"User-Agent": USER_AGENT}

In [10]:
response = requests.get(url, headers=headers)

In [11]:
result = response.json()

In [12]:
for story in result['data']['children']: 
    print(story['data']['title'])

South Korea legalize abortion after 66 year ban
Fecal transplants result in massive long-term reduction in autism symptoms
Declassified documents show CIA knew of secret Latin assassin unit - With the blessing of a CIA bent on thwarting Soviet expansion, South American military juntas together formed a special unit charged with going to France and elsewhere abroad to exterminate leftist opposition leaders
Oil-eating bacteria discovered at the bottom of the Mariana Trench
Chomsky: Trump Radically Interfered with Israel’s Election to Help Re-elect Netanyahu
Man wearing Trump T-shirt admits abusing worshippers at New Zealand mosque
'Extraordinary' 500-year-old library catalogue reveals books lost to time
‘Attenborough effect’ leads to 53% drop in single use plastic in 12 months
Brazilian president raises eyebrows by saying the Holocaust can be forgiven
One in five cases of childhood asthma tied to pollution from cars and trucks, global study finds
Chinese Hacking Steals Billions; U.S. Bus

In [13]:
from time import sleep

In [14]:
def get_links(subreddit, token, n_pages=5):
    stories = []
    after = None
    for page_number in range(n_pages):
        # Sleep before making calls to avoid going over the API limit
        sleep(2)
        # Setup headers and make call, just like in the login function
        headers = {"Authorization": "bearer {}".format(token['access_token']), "User-Agent": USER_AGENT} 
        url = "https://oauth.reddit.com/r/{}?limit=100". format(subreddit)
        if after:
            # Append cursor for next page, if we have one
            url += "&after={}".format(after)
        response = requests.get(url, headers=headers)
        result = response.json()
        # Get the new cursor for the next loop
        after = result['data']['after']
        # Add all of the news items to our stories list
        for story in result['data']['children']:
            stories.append((story['data']['title'], story['data']['url'], story['data']['score']))
    return stories

In [15]:
stories = get_links("worldnews", token)

In [16]:
import os 
data_folder = os.path.join(os.path.expanduser("~"),"Datamining","chapter10","Data", "websites", "raw")

In [17]:
import hashlib

In [18]:
number_errors = 0


In [ ]:
for title, url, score in stories:
    output_filename = hashlib.md5(url.encode()).hexdigest() 
    fullpath = os.path.join(data_folder, output_filename + ".txt")
    try: 
        response = requests.get(url) 
        data = response.text 
        with open(fullpath, 'w') as outf: 
            outf.write(data)
    except Exception as e:
        number_errors += 1
        # You can use this to view the errors, if you are getting too many:
        #raise

In [19]:
number_errors

0

In [20]:
filenames = [os.path.join(data_folder, filename) for filename in os.listdir(data_folder)]

In [21]:
text_output_folder = os.path.join(os.path.expanduser("~"),"Datamining","chapter10","Data", "websites", "textonly")

In [22]:
import lxml
from lxml import etree

In [23]:
skip_node_types = ["script", "head", "style", etree.Comment]


In [24]:
parser = etree.HTMLParser()

def get_text_from_file(filename):
    with open(filename) as inf:
        html_tree = etree.parse(inf, parser) 
    return get_text_from_node(html_tree.getroot())

In [30]:
def get_text_from_node(node):
    try:
        if len(node) == 0: 
        # No children, just return text from this item
            if node.text: 
                return node.text 
            else:
                return ""
        else:
            # This node has children, return the text from it:
            results = (get_text_from_node(child)
                       for child in node
                       if child.tag not in skip_node_types)
        result = str.join("\n", (r for r in results if len(r) > 1))
        if len(result) >= 100:
            return result
        else:
            return ""
    except:
        return ""

In [31]:
for filename in os.listdir(data_folder):
    text = get_text_from_file(os.path.join(data_folder, filename)) 
    with open(os.path.join(text_output_folder, filename), 'w' , encoding='utf-8') as outf: 
        outf.write(text)

In [32]:
from sklearn.cluster import KMeans

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
from sklearn.pipeline import Pipeline
n_clusters = 10 
pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4)),
                                     ('clusterer', KMeans(n_clusters=n_clusters)) ])

In [36]:

documents = [open(os.path.join(text_output_folder, filename), encoding='utf-8').read() for filename in os.listdir(text_output_folder)]

In [37]:
pipeline.fit(documents)
labels = pipeline.predict(documents)

In [38]:
from collections import Counter
c = Counter(labels) 
for cluster_number in range(n_clusters): 
    print("Cluster {} contains {} samples".format(cluster_number, c[cluster_number]))

Cluster 0 contains 138 samples
Cluster 1 contains 5 samples
Cluster 2 contains 2 samples
Cluster 3 contains 1 samples
Cluster 4 contains 7 samples
Cluster 5 contains 2 samples
Cluster 6 contains 1 samples
Cluster 7 contains 2 samples
Cluster 8 contains 2 samples
Cluster 9 contains 1 samples


In [39]:
pipeline.named_steps['clusterer'].inertia_

69.51385148898275

In [40]:
inertia_scores = [] 
n_cluster_values = list(range(2, 20)) 
for n_clusters in n_cluster_values: 
    cur_inertia_scores = [] 
    X = TfidfVectorizer(max_df=0.4).fit_transform(documents) 
    for i in range(10): 
        km = KMeans(n_clusters=n_clusters).fit(X) 
        cur_inertia_scores.append(km.inertia_) 
        inertia_scores.append(cur_inertia_scores)

In [41]:
n_clusters

19

In [42]:
n_clusters = 6 
pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4)),
                     ('clusterer', KMeans(n_clusters=n_clusters)) ])
pipeline.fit(documents) 
labels = pipeline.predict(documents)

In [43]:
n_clusters

6

In [44]:
terms = pipeline.named_steps['feature_extraction'].get_feature_names()

In [45]:
c = Counter(labels)

In [46]:
for cluster_number in range(n_clusters): 
    print("Cluster {} contains {} samples".format(cluster_number, c[cluster_number]))
    print(" Most important terms")
    centroid = pipeline.named_steps['clusterer'].cluster_centers_[cluster_number]
    most_important = centroid.argsort()
    for i in range(5):
        term_index = most_important[-(i+1)]
        print(" {0}) {1} (score: {2:.4f})".format(i+1, terms[term_index], centroid[term_index]))

Cluster 0 contains 145 samples
 Most important terms
 1) we (score: 0.0219)
 2) by (score: 0.0213)
 3) he (score: 0.0213)
 4) have (score: 0.0205)
 5) they (score: 0.0180)
Cluster 1 contains 7 samples
 Most important terms
 1) cache (score: 0.4499)
 2) found (score: 0.3069)
 3) ewr (score: 0.2317)
 4) varnish (score: 0.2317)
 5) 404 (score: 0.2317)
Cluster 2 contains 1 samples
 Most important terms
 1) electric (score: 0.4193)
 2) australia (score: 0.3478)
 3) vehicles (score: 0.2375)
 4) australians (score: 0.1880)
 5) voters (score: 0.1732)
Cluster 3 contains 2 samples
 Most important terms
 1) trump (score: 0.2739)
 2) kim (score: 0.2718)
 3) korea (score: 0.2582)
 4) north (score: 0.2276)
 5) summit (score: 0.1931)
Cluster 4 contains 5 samples
 Most important terms
 1) police (score: 0.3267)
 2) car (score: 0.2714)
 3) officers (score: 0.2596)
 4) embassy (score: 0.1958)
 5) ambassador (score: 0.1570)
Cluster 5 contains 1 samples
 Most important terms
 1) sudan (score: 0.2998)
 2) 

In [47]:
X = pipeline.transform(documents)

In [48]:
from scipy.sparse import csr_matrix

In [49]:
import numpy as np
def create_coassociation_matrix(labels):
    rows = [] 
    cols = []
    unique_labels = set(labels) 
    for label in unique_labels:
        indices = np.where(labels == label)[0]
        for index1 in indices:
            for index2 in indices:
                rows.append(index1)
                cols.append(index2)
    data = np.ones((len(rows),)) 
    return csr_matrix((data, (rows, cols)), dtype='float')

In [50]:
C = create_coassociation_matrix(labels)

In [51]:
C

<161x161 sparse matrix of type '<class 'numpy.float64'>'
	with 21105 stored elements in Compressed Sparse Row format>

In [52]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [53]:
mst = minimum_spanning_tree(C)

In [54]:
mst = minimum_spanning_tree(-C)

In [55]:
pipeline.fit(documents) 
labels2 = pipeline.predict(documents) 
C2 = create_coassociation_matrix(labels2) 
C_sum = (C + C2) / 2

In [56]:
mst = minimum_spanning_tree(-C_sum) 
mst.data[mst.data > -1] = 0

In [57]:
from scipy.sparse.csgraph import connected_components 
number_of_clusters, labels = connected_components(mst)

In [58]:
from sklearn.base import BaseEstimator, ClusterMixin 
class EAC(BaseEstimator, ClusterMixin):
    def __init__(self, n_clusterings=10, cut_threshold=0.5, n_clusters_range=(3, 10)): 
        self.n_clusterings = n_clusterings
        self.cut_threshold = cut_threshold
        self.n_clusters_range = n_clusters_range

    def fit(self, X, y=None):
        C = sum((create_coassociation_matrix(self._single_clustering(X))
                 for i in range(self.n_clusterings)))
        mst = minimum_spanning_tree(-C)
        mst.data[mst.data > -self.cut_threshold] = 0
        mst.eliminate_zeros()
        self.n_components, self.labels_ = connected_components(mst)
        return self
 
    def _single_clustering(self, X):
        n_clusters = np.random.randint(*self.n_clusters_range)
        km = KMeans(n_clusters=n_clusters)
        return km.fit_predict(X)
 
    def fit_predict(self, X):
        self.fit(X)
        return self.labels_

In [59]:
pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4)),
                     ('clusterer', EAC()) ])

In [60]:
vec = TfidfVectorizer(max_df=0.4) 
X = vec.fit_transform(documents)

In [61]:
from sklearn.cluster import MiniBatchKMeans 
mbkm = MiniBatchKMeans(random_state=14, n_clusters=3)

In [62]:
batch_size = 10 
for iteration in range(int(X.shape[0] / batch_size)): 
    start = batch_size * iteration 
    end = batch_size * (iteration + 1) 
    mbkm.partial_fit(X[start:end])

In [63]:
labels = mbkm.predict(X)

In [64]:
labels

array([0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0])

In [65]:
class PartialFitPipeline(Pipeline):
    def partial_fit(self, X, y=None):
        Xt = X
        for name, transform in self.steps[:-1]:
            Xt = transform.transform(Xt)
        return self.steps[-1][1].partial_fit(Xt, y=y)

In [66]:
from sklearn.feature_extraction.text import HashingVectorizer

pipeline = PartialFitPipeline([('feature_extraction', HashingVectorizer()),
                               ('clusterer', MiniBatchKMeans(random_state=14, n_clusters=3)) ])
batch_size = 10 
for iteration in range(int(len(documents) / batch_size)): 
    start = batch_size * iteration 
    end = batch_size * (iteration + 1)
    pipeline.partial_fit(documents[start:end]) 
labels = pipeline.predict(documents)

In [67]:
labels

array([1, 1, 1, 2, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0])